In [1]:
import pandas as pd
import numpy as np

In [2]:
# 데이터 불러오기
df_raw = pd.read_csv("../data/PatientInfo.csv")
df_raw.info()
df_raw.age.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4004 entries, 0 to 4003
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   patient_id          4004 non-null   int64  
 1   global_num          2806 non-null   float64
 2   sex                 3674 non-null   object 
 3   birth_year          3267 non-null   object 
 4   age                 3525 non-null   object 
 5   country             4004 non-null   object 
 6   province            4004 non-null   object 
 7   city                3926 non-null   object 
 8   disease             18 non-null     object 
 9   infection_case      3211 non-null   object 
 10  infection_order     31 non-null     float64
 11  infected_by         899 non-null    object 
 12  contact_number      680 non-null    object 
 13  symptom_onset_date  535 non-null    object 
 14  confirmed_date      4001 non-null   object 
 15  released_date       1508 non-null   object 
 16  deceas

20s     860
50s     625
30s     493
40s     483
60s     423
70s     210
80s     162
10s     160
0s       60
90s      47
30        1
100s      1
Name: age, dtype: int64

In [3]:
# birth_year를 통해 age의 결측치 일부 회복
df_raw.age = df_raw.age.replace('s','',regex=True)
df_raw.age = pd.to_numeric(df_raw.age)
for i in range(len(df_raw)):
    if df_raw.birth_year[i] == " ":
        df_raw.birth_year[i] = np.NaN
    else:
        df_raw.birth_year[i] = pd.to_numeric(df_raw.birth_year[i])
for i in range(len(df_raw)):
    if pd.notna(df_raw.birth_year[i]) and pd.isna(df_raw.age[i]):
        df_raw.age[i]=(2020-df_raw.birth_year[i])//10*10

C:\Users\HDK\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\HDK\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\HDK\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [4]:
# age, infected_by에서 결측치 제거한 결과물을 df_refine 생성
df_refine = df_raw.dropna(subset = ['age', 'infected_by'])

# 필요한 컬럼들을 시리즈로 추출
patient_id = df_refine.patient_id
age = df_refine.age
infection_case = df_refine.infection_case
infected_by = df_refine.infected_by

# 시리즈가 된 patient_id, age, infection_case, infected_by를 DF로 통합
df_refine = pd.concat([patient_id, age, infection_case, infected_by], axis=1)

# infection_case 컬럼에서 값이 'contact with patient'인 데이터만 남기고 나머지는 전부 제거
df_refine = df_refine.loc[df_refine['infection_case'].str.contains('contact with patient', na=False)]

df_refine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 808 entries, 2 to 4000
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   patient_id      808 non-null    int64  
 1   age             808 non-null    float64
 2   infection_case  808 non-null    object 
 3   infected_by     808 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 31.6+ KB


In [5]:
# 노년층 감염자들만을 모은 DF인 df_more_than_80 생성
df_more_than_80 = df_refine[df_refine.age >= 80.0]
df_more_than_80 = df_more_than_80.reset_index(drop=True, inplace=False)
df_more_than_80['infected_by'] = pd.to_numeric(df_more_than_80['infected_by'])

df_more_than_80

,patient_id,age,infection_case,infected_by
0,1000000013,80.0,contact with patient,1000000017
1,1000000021,80.0,contact with patient,1000000016
2,1000000284,80.0,contact with patient,1000000279
3,1000000517,80.0,contact with patient,1000000509
4,1100000087,80.0,contact with patient,1100000002
5,1300000012,80.0,contact with patient,1300000011
6,1500000020,80.0,contact with patient,1500000019
7,2000000292,80.0,contact with patient,2000000236
8,2000000335,80.0,contact with patient,2000000309
9,2000000378,90.0,contact with patient,2000000309


In [6]:
df_merged = df_more_than_80.merge(right=df_raw,left_on='infected_by',right_on='patient_id')
target_ages = df_merged['age_y'].value_counts()
target_ages.name = 'target_ages'

In [7]:
# 각 연령대를 연령층으로 그룹화
age_20_to_30 = target_ages[20.0]
age_40_to_50 = target_ages[40.0] + target_ages[50.0]
age_60_to_70 = target_ages[60.0] + target_ages[70.0]
age_over_70 = target_ages[80.0] + target_ages[90.0]

In [8]:
dict_age_group = {'청년':age_20_to_30, '중년':age_40_to_50, '장년':age_60_to_70, '노년':age_over_70}
ser_age_group = pd.Series(dict_age_group)
ser_age_group

청년     3
중년     4
장년    13
노년    19
dtype: int64

In [9]:
import plotly.graph_objects as go
# import plotly.express as px

In [11]:
# 파이차트로 결과물 시각화
fig = go.Figure(data=[go.Pie(labels=ser_age_group.index, values=ser_age_group, title="노년층 감염을 일으킨 감염자들의 주 연령층")])
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()